In [1]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [2]:
data = np.loadtxt('train.csv', delimiter=',', skiprows=1)  # skiprows=1 si hay encabezados
# print(data)
X = data[:, :-1]  # Todas las columnas excepto la última son las características
y = data[:, -1]    # La última columna es la variable objetivo
print(X.shape)
print(y.shape)

(50000, 3072)
(50000,)


In [3]:
y

array([6., 9., 9., ..., 9., 1., 1.])

In [4]:
# data.shape
X.shape[1]

3072

In [5]:
input_layer_size  = 3072
num_labels = 10
m = y.size

In [6]:
def calcularSigmoide(z):
    # Calcula la sigmoide de z.

    return 1.0 / (1.0 + np.exp(-z))

In [7]:
def calcularCosto(theta, X, y, lambda_):
    # Inicializa algunos valores utiles
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = calcularSigmoide(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    # Se aplica regularizacion en la siguiente linea
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [8]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate((np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0), axis=1)
print(X_t)
# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

[[1.  0.1 0.6 1.1]
 [1.  0.2 0.7 1.2]
 [1.  0.3 0.8 1.3]
 [1.  0.4 0.9 1.4]
 [1.  0.5 1.  1.5]]


In [9]:
J, grad = calcularCosto(theta_t, X_t, y_t, lambda_t)

print('Costo         : {:.6f}'.format(J))
print('Costo esperadot: 2.534819')
print('-----------------------')
print('Gradientes:')
print(' [{:.6f}, {:.6f}, {:.6f}, {:.6f}]'.format(*grad))
print('Gradientes esperados:')
print(' [0.146561, -0.548558, 0.724722, 1.398003]');

Costo         : 2.534819
Costo esperadot: 2.534819
-----------------------
Gradientes:
 [0.146561, -0.548558, 0.724722, 1.398003]
Gradientes esperados:
 [0.146561, -0.548558, 0.724722, 1.398003]


In [10]:
def oneVsAll(X, y, num_labels, lambda_):

    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)


    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(calcularCosto,
                                initial_theta,
                                (X, (y == (c + 1)), lambda_),
                                jac=True,
                                method='BFGS',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [11]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)

C:\Users\DIEGO\AppData\Local\Temp\ipykernel_16612\3442295675.py:4: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))
C:\Users\DIEGO\AppData\Local\Temp\ipykernel_16612\2753339385.py:17: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


In [12]:
print(all_theta)

[[-1.72801130e-03  1.37743941e-03  2.54248328e-04 ...  1.77829119e-03
   2.77407275e-04  6.41383016e-04]
 [-2.21314505e-03  1.69687268e-03  2.30056345e-03 ...  3.53694922e-04
  -3.18712339e-03 -1.48370055e-03]
 [-2.13162779e-03  2.57588805e-04  1.67070203e-03 ...  1.80436019e-04
   5.59581921e-04  1.29213713e-03]
 ...
 [-4.05444462e-03  2.81277662e-04 -2.22022804e-04 ...  1.31772585e-03
   2.07572173e-03  1.01720033e-03]
 [-2.11005098e-03  5.30735078e-04 -2.00559356e-04 ... -1.55558892e-04
   2.05832355e-03  1.76962217e-03]
 [-5.91629343e-06 -7.73323093e-04 -7.69948557e-04 ... -6.73455113e-04
  -6.73902740e-04 -6.76716647e-04]]


In [13]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)

    return p + 1

In [14]:
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[128:141, :].copy()
yPrueba = y[128:141].copy()
# XPrueba = [[11.44, 2.3, 2.41, 20, 96, 2.9, 2.79, 0.82, 1.83, 3.25, 0.8, 3.39, 625]]
# yPrueba = [2]
print(XPrueba.shape)
print(len(XPrueba))
# print(XPrueba)
# print(yPrueba.shape)
# print(yPrueba)

#print(np.ones((1)))
#p = np.zeros(1)
XPrueba = np.concatenate((np.ones((13, 1)), XPrueba), axis=1)
print(XPrueba)
# print(XPrueba.shape)
# print(XPrueba)
p = np.argmax(calcularSigmoide(XPrueba.dot(all_theta.T)), axis = 1)
print(p + 1)

(50000, 3072)
Precision del conjuto de entrenamiento: 43.11%
(13, 3072)
13
[[  1.  25.  32. ...  77.  76.  72.]
 [  1. 104. 106. ... 195. 196. 223.]
 [  1. 128. 120. ... 102. 114. 113.]
 ...
 [  1. 205. 200. ... 219. 208. 208.]
 [  1. 236. 231. ...  94.  97. 114.]
 [  1. 185. 184. ...  99.  99.  96.]]
[3 2 4 4 2 7 1 8 1 8 2 8 7]
